# CoolPropで乾球温度・湿球温度から各種湿り空気特性を計算するノートブック

このノートブックは、**CoolProp** の `HAPropsSI` 関数を用いて、入力した乾球温度と湿球温度（必要に応じて圧力）から次の量を計算します：

- 相対湿度 (RH)
- 絶対湿度（含湿量: kg/kg(DA))
- 比エンタルピー (kJ/kg(DA))
- 比容積 (m³/kg(DA))
- 露点温度 (°C)

> **注**: CoolProp の湿り空気 API `HAPropsSI` は SI 単位系で動作します。入力は乾球温度 `T` (K)、圧力 `P` (Pa)、湿球温度はキー `Twb`（別名 `B`, `T_wb`, `WetBulb`）を用います。出力キーには `R` (相対湿度), `W` (含湿量), `H` (比エンタルピー: J/kg(DA)), `V` (比容積: m³/kg(DA)), `Tdp` (露点温度: K) などがあります。

実行前に、必要なら以下で CoolProp をインストールしてください：
```bash
pip install CoolProp
```

参考文献：Bell, I. H., Wronski, J., Quoilin, S., & Lemort, V. (2014). Pure and Pseudo-pure Fluid Thermophysical Property Evaluation and the Open-Source Thermophysical Propertiy Library CoolProp. *Industrial & Engineering Chemistry Research*, **53**, 2498-2508. DOI:10.1021/ie4033999

In [1]:
# -*- coding: utf-8 -*-

# 必要なライブラリをインポート
from CoolProp.HumidAirProp import HAPropsSI
import CoolProp

def psychrometrics_from_dry_and_wet_bulb(Tdry_C: float, Twet_C: float, P_Pa: float = 101325.0):
    """
    乾球温度(°C)と湿球温度(°C)、圧力(Pa)から以下を計算します。

    - 相対湿度 (%)
    - 絶対湿度（含湿量: kg/kg(DA)）
    - 比エンタルピー (kJ/kg(DA))
    - 比容積 (m³/kg(DA))
    - 露点温度 (°C)

    参考: CoolProp の HAPropsSI は SI 単位系で、"T"(K), "P"(Pa), "Twb"(K) の入力から
    "R", "W", "H", "V", "Tdp" などを返します。

    パラメータ:
        Tdry_C: 乾球温度 [°C]
        Twet_C: 湿球温度 [°C]
        P_Pa  : 全圧 [Pa] (既定: 101325)

    返り値:
        dict で各物性値を返します。
    """
    if P_Pa <= 0:
        raise ValueError("圧力 P_Pa は正の値にしてください。")

    # SI 単位に変換
    Tdry_K = Tdry_C + 273.15
    Twet_K = Twet_C + 273.15

    # 入力チェック: 一般に湿球温度が乾球温度より高いと不正
    if Twet_K > Tdry_K + 1e-6:
        raise ValueError("湿球温度が乾球温度より高い場合、計算は不正になる可能性があります。入力を確認してください。")

    # ===== CoolProp の HAPropsSI を用いた計算 =====
    RH = HAPropsSI('R',   'T', Tdry_K, 'P', P_Pa, 'Twb', Twet_K)       # 相対湿度(0-1)
    W  = HAPropsSI('W',   'T', Tdry_K, 'P', P_Pa, 'Twb', Twet_K)       # 絶対湿度(kg/kg(DA))
    H  = HAPropsSI('H',   'T', Tdry_K, 'P', P_Pa, 'Twb', Twet_K)       # 比エンタルピー(J/kg(DA))
    V  = HAPropsSI('V',   'T', Tdry_K, 'P', P_Pa, 'Twb', Twet_K)       # 比容積(m³/kg(DA))
    Tdp_K = HAPropsSI('Tdp', 'T', Tdry_K, 'P', P_Pa, 'Twb', Twet_K)    # 露点温度(K)

    # 追加の便利な量

    return {
        'CoolProp_version': getattr(CoolProp, '__version__', 'unknown'),
        '相対湿度_%': RH * 100.0,
        '絶対湿度_kg_per_kgDA': W,
        '比エンタルピー_kJ_per_kgDA': H / 1000.0,
        '比容積_m3_per_kgDA': V,
        '露点温度_°C': Tdp_K - 273.15,
    }

def pretty_print_results(results: dict):
    """計算結果を整形して表示"""
    print(f"CoolProp バージョン: {results.get('CoolProp_version','')}")
    print("— 計算結果 —")
    print(f"相対湿度: {results['相対湿度_%']:.3f} %")
    print(f"絶対湿度: {results['絶対湿度_kg_per_kgDA']:.3f} kg/kg(DA)")
    print(f"比エンタルピー: {results['比エンタルピー_kJ_per_kgDA']:.3f} kJ/kg(DA)")
    print(f"比容積: {results['比容積_m3_per_kgDA']:.6f} m³/kg(DA)")
    print(f"露点温度: {results['露点温度_°C']:.3f} °C")


## 使い方
下のセルの `Tdry_C`（乾球温度 °C）, `Twet_C`（湿球温度 °C）, `P_Pa`（圧力 Pa）を設定して実行してください。
圧力を省略した場合は標準大気圧 101325 Pa を使用します。


In [2]:
# 入力例
Tdry_C = 35.0   # 乾球温度 [°C]
Twet_C = 25.0   # 湿球温度 [°C]
P_Pa   = 101325 # 圧力 [Pa]

try:
    results = psychrometrics_from_dry_and_wet_bulb(Tdry_C, Twet_C, P_Pa)
    pretty_print_results(results)
except Exception as e:
    print("計算エラー:", e)


CoolProp バージョン: 7.0.0
— 計算結果 —
相対湿度: 44.735 %
絶対湿度: 0.016 kg/kg(DA)
比エンタルピー: 76.059 kJ/kg(DA)
比容積: 0.895028 m³/kg(DA)
露点温度: 21.201 °C


## メモ（定義と単位）
- **相対湿度** `R`: 水蒸気のモル分率を飽和時のモル分率で割った値（0〜1）。出力は % に変換して表示しています。
- **絶対湿度** `W`: 乾き空気 1 kg あたりの水蒸気質量 [kg/kg(DA)]。表示は kg/kg(DA)。
- **比エンタルピー** `H`: 湿り空気の比エンタルピー（乾き空気あたり）[J/kg(DA)]。表示は kJ/kg(DA)。
- **比容積** `V`: 湿り空気の比容積（乾き空気あたり）[m³/kg(DA)]。
- **露点温度** `Tdp`: 飽和に達する温度 [K]。表示は °C。

> `HAPropsSI` では湿球温度入力に `Twb`（別名 `B`, `T_wb`, `WetBulb`）を指定できます。入力は `T`(乾球温度), `P`(圧力) と、湿度の指定（`R`/`W`/`Twb`/`Tdp` のいずれか）で状態が定まります。
